In [1]:
%pip3 install -U sentence-transformers
%pip3 install lyricsgenius
%pip install python-dotenv openai

In [2]:
# Imports and setup
import pandas as pd
import numpy as np
import torch
import requests
import lyricsgenius
import dotenv 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from collections import Counter
from nltk.probability import FreqDist
import logging
from langdetect import detect
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from dotenv import load_dotenv
import os
import openai
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/anita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/anita/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '1075d2afec3d414ca02ec7002f1d5576'
client_secret = 'bb1fd201730b484983d952ecbd344b33'
credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=credentials)

### Clean code 

In [4]:
df = pd.read_csv('/Users/anita/Documents/NLPSongs-/music_data.csv')  
print("Data loaded successfully!")
print("Sample data:")
df

Data loaded successfully!
Sample data:


,artist,title,id,release_date,lyrics
0,Ariana Grande,"​thank u, next",4063065,2018-11-03,[Verse 1]\nThought I'd end up with Sean\nBut h...
1,Ariana Grande,7 rings,4067762,2019-01-18,"[Verse 1]\nYeah, breakfast at Tiffany's and bo..."
2,Ariana Grande,​God is a woman,3681280,2018-07-13,"[Chorus]\nYou, you love it how I move you\nYou..."
3,Ariana Grande,Side To Side,2457495,2016-05-20,[Intro: Ariana Grande & Nicki Minaj]\nI've bee...
4,Ariana Grande,​​no tears left to cry,3649172,2018-04-20,"[Intro]\nRight now, I'm in a state of mind\nI ..."
...,...,...,...,...,...
3739,Taylor Swift,"Taylor Swift - no body, no crime (Traducción a...",6274728,2020-12-11,[Intro: HAIM]\nÉl lo hizo\nÉl lo hizo\n\n[Vers...
3740,Taylor Swift,Welcome Back Grunwald,6226864,NaN,Turn WYCD on\nYou're\nOn your Grunwald\nBack f...
3741,Taylor Swift,Tolerate it (Polskie Tłumaczenie),6315848,2020-12-11,[Zwrotka 1]\nSiedzę i patrzę jak czytasz z gło...
3742,Taylor Swift,Find you,6209316,NaN,Trying just like they say\nJust taking the ste...


In [5]:
df['artist'].unique()

array(['Ariana Grande', 'Beyoncé', 'Billie Eilish', 'Cardi B',
       'Charlie Puth', 'Coldplay', 'Drake', 'Dua Lipa', 'Ed Sheeran',
       'Eminem', 'Justin Bieber', 'Katy Perry', 'Khalid', 'Lady Gaga',
       'Maroon 5', 'Nicki Minaj', 'Post Malone', 'Rihanna',
       'Selena Gomez', 'Taylor Swift'], dtype=object)

In [6]:
null_values = df.isnull().sum()
print(null_values)

artist            0
title             0
id                0
release_date    993
lyrics            0
dtype: int64


In [7]:
#Remove null values
df= df.dropna()

In [8]:
duplicate_rows = df.duplicated().sum()
print(duplicate_rows)

0


In [9]:
df['title'].unique()

array(['\u200bthank u, next', '7 rings', '\u200bGod is a woman', ...,
       'Taylor Swift - no body, no crime (Traducción al español)',
       'Tolerate it (Polskie Tłumaczenie)', 'Clean - Türkçe Çeviri'],
      dtype=object)

In [10]:
df['lyrics'].unique()

array(['[Verse 1]\nThought I\'d end up with Sean\nBut he wasn\'t a match\nWrote some songs about Ricky\nNow I listen and laugh\nEven almost got married\nAnd for Pete, I\'m so thankful\nWish I could say, "Thank you" to Malcolm\n\'Cause he was an angel\n\n[Pre-Chorus]\nOne taught me love\nOne taught me patience\nAnd one taught me pain\nNow, I\'m so amazing\nSay I\'ve loved and I\'ve lost\nBut that\'s not what I see\nSo, look what I got\nLook what you taught me\nAnd for that, I say\n\n[Chorus]\nThank you, next (Next)\nThank you, next (Next)\nThank you, next\nI\'m so fuckin\' grateful for my ex\nThank you, next (Next)\nThank you, next (Next)\nThank you, next (Next)\nI\'m so fuckin\'—\n\n[Verse 2]\nSpend more time with my friends\nI ain\'t worried \'bout nothin\'\nPlus, I met someone else\nWe havin\' better discussions\nI know they say I move on too fast\nBut this one gon\' last\n\'Cause her name is Ari\nAnd I\'m so good with that (So good with that)\n\n[Pre-Chorus]\nShe taught me love (Lov

In [11]:
# Regex pattern that removes any characters that are not in the range of all basic English letters, numbers and commun punctuation
pattern = r'[^\x00-\x7F]+'

mask = df['title'].str.contains(pattern) | df['lyrics'].str.contains(pattern)

df = df[~mask]

In [12]:
df['title'].unique()

array(['7 rings', 'Dangerous Woman', 'Into You', ..., 'Playlist by ME!',
       'Barnyard Song', 'Rare Peer (Brenda Lee Tribute)'], dtype=object)

In [13]:
df['lyrics'].unique()

array(['[Verse 1]\nYeah, breakfast at Tiffany\'s and bottles of bubbles\nGirls with tattoos who like getting in trouble\nLashes and diamonds, ATM machines\nBuy myself all of my favorite things (Yeah)\nBeen through some bad shit, I should be a sad bitch\nWho woulda thought it\'d turn me to a savage?\nRather be tied up with calls and not strings\nWrite my own checks like I write what I sing, yeah (Yeah)\n\n[Pre-Chorus]\nMy wrist, stop watchin\', my neck is flossy\nMake big deposits, my gloss is poppin\'\nYou like my hair? Gee, thanks, just bought it\nI see it, I like it, I want it, I got it (Yeah)\n\n[Chorus]\nI want it, I got it, I want it, I got it\nI want it, I got it, I want it, I got it\nYou like my hair? Gee, thanks, just bought it\nI see it, I like it, I want it, I got it (Yep)\n\n[Verse 2]\nWearing a ring, but ain\'t gon\' be no "Mrs."\nBought matching diamonds for six of my bitches\nI\'d rather spoil all my friends with my riches\nThink retail therapy my new addiction\nWhoever s

### Preprocessing 

In [14]:
# Utility functions
# Expanding Contractions
CONTRACTION_MAP = {
    "can't": "cannot",
    "won't": "will not",
    "n't": " not",
    "'re": " are",
    "'s": " is",
    "'m": " am",
    "'ll": " will",
    "'d": " would",
    "'ve": " have",
    "o'clock": "of the clock",
    "ma'am": "madam",
    "let's": "let us"
}

def expand_contractions(lyrics, contraction_mapping=CONTRACTION_MAP):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match.lower() if match.lower() in contraction_mapping else match)
        return first_char + expanded_contraction[1:]
    expanded_lyrics = contractions_pattern.sub(expand_match, lyrics)
    return expanded_lyrics

def preprocess_lyrics(lyrics):
    lyrics = expand_contractions(lyrics)  
    lyrics = re.sub(r'\[(.*?)\]', '', lyrics) 
    lyrics = re.sub(r'\W+', ' ', lyrics)  # remove anything that is not a word
    lyrics = lyrics.lower() 
    tokens = word_tokenize(lyrics)  
    return ' '.join(tokens)  

#### Encode Lyrics

In [15]:
logging.basicConfig(level=logging.INFO)
logging.info("Starting to preprocess lyrics")
df['preprocessed_lyrics'] = df['lyrics'].apply(preprocess_lyrics)
logging.info("Finished preprocessing lyrics")

INFO:root:Starting to preprocess lyrics
INFO:root:Finished preprocessing lyrics


In [16]:
df[['lyrics', 'preprocessed_lyrics']].head(10)

,lyrics,preprocessed_lyrics
1,"[Verse 1]\nYeah, breakfast at Tiffany's and bo...",yeah breakfast at tiffany is and bottles of bu...
19,"[Intro]\n(Oh, yeah)\n\n[Verse 1]\nDon't need p...",oh yeah donnot need permission made my decisio...
21,"[Verse 1]\nI'm so into you, I can barely breat...",i am so into you i can barely breathe and all ...
25,"[Verse 1]\nI was a liar, I gave in to the fire...",i was a liar i gave in to the fire i know i sh...
31,[Intro: Iggy Azalea & Ariana Grande]\nUh-huh! ...,uh huh it is iggy iggs i got one more problem ...
33,[Verse 1]\nHey\nI know what I came to do\nAnd ...,hey i know what i came to do and that ainnot g...
34,"[Verse 1: Ariana Grande]\nAnytime I'm alone, I...",anytime i am alone i can not help thinking abo...
40,[Verse 1]\nMidnight shadows\nWhen finding love...,midnight shadows when finding love is a battle...
48,[Verse 1]\nI cried enough tears to see my own ...,i cried enough tears to see my own reflection ...
50,[Verse 1]\nHow do I make the phone ring?\nWhy ...,how do i make the phone ring why do i even car...


The text-embedding-ada-002 model from OpenAI is a text embedding model designed to convert text into high-dimensional vectors. These vectors can be used in various applications such as semantic search, clustering, text similarity, and more. The model is part of the family of models developed by OpenAI, and it is optimized for both performance and efficiency.

Like other models from OpenAI, it is trained on a diverse and large corpus of text data. This training helps the model understand a wide variety of topics and contexts.

In [17]:
def get_openai_embeddings(texts):
    # Generate embeddings using OpenAI's API
    response = openai.Embedding.create(
        input=texts,
        engine="text-embedding-ada-002",  
        user=api_key
    )
    # Extract embeddings from the response
    embeddings = [embedding['embedding'] for embedding in response['data']]
    return embeddings

In [18]:
logging.info("Starting to get embeddings")
df['embeddings'] = list(get_openai_embeddings(df['preprocessed_lyrics'].tolist()))
logging.info("Finished getting embeddings")

INFO:root:Starting to get embeddings
INFO:root:Finished getting embeddings


In [19]:
df[['artist', 'preprocessed_lyrics', 'embeddings']].head(10)

,artist,preprocessed_lyrics,embeddings
1,Ariana Grande,yeah breakfast at tiffany is and bottles of bu...,"[-0.03643781691789627, -0.013206989504396915, ..."
19,Ariana Grande,oh yeah donnot need permission made my decisio...,"[-0.010308732278645039, -0.014625722542405128,..."
21,Ariana Grande,i am so into you i can barely breathe and all ...,"[-0.03010842576622963, -0.013051347807049751, ..."
25,Ariana Grande,i was a liar i gave in to the fire i know i sh...,"[-0.021388838067650795, -0.0195090901106596, -..."
31,Ariana Grande,uh huh it is iggy iggs i got one more problem ...,"[-0.016048762947320938, 0.000931991555262357, ..."
33,Ariana Grande,hey i know what i came to do and that ainnot g...,"[-0.027668043971061707, -0.025612173601984978,..."
34,Ariana Grande,anytime i am alone i can not help thinking abo...,"[-0.01779363863170147, -0.005605586338788271, ..."
40,Ariana Grande,midnight shadows when finding love is a battle...,"[0.01216826494783163, -0.019625365734100342, 0..."
48,Ariana Grande,i cried enough tears to see my own reflection ...,"[-0.001736154081299901, -0.021960001438856125,..."
50,Ariana Grande,how do i make the phone ring why do i even car...,"[-0.02340000495314598, -0.013523552566766739, ..."


In [20]:
def search_lyrics_sentence_transformer(query, df, get_openai_embeddings):
    query_embedding = get_openai_embeddings([query])[0]
    df['embeddings'] = get_openai_embeddings(df['lyrics'].tolist())
    
    similarities = cosine_similarity([query_embedding], df['embeddings'].tolist()).flatten()
    top_indices = similarities.argsort()[-5:][::-1]
    return df.iloc[top_indices]

In [21]:
def search_spotify_from_results(song_title, artist):
    query = f"track:{song_title} artist:{artist}"
    results = spotify.search(q=query, type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'album_image_url': track['album']['images'][0]['url'] if track['album']['images'] else None,
            'preview_url': track['preview_url'],
            'spotify_url': track['external_urls']['spotify']
        }
    else:
        return "No results found on Spotify."

### Search

In [22]:
def display_spotify_details(spotify_info):
    if isinstance(spotify_info, dict):
        html_content = f"""
        <div style="border: 1px solid #ccc; padding: 10px; border-radius: 5px; display: flex; align-items: center; gap: 20px; background-color: #f9f9f9;">
            <img src="{spotify_info.get('album_image_url', '')}" style="width: 100px; height: auto; border-radius: 5px;">
            <div>
                <h3>{spotify_info['name']} - {spotify_info['artist']}</h3>
                <p><strong>Album:</strong> {spotify_info['album']}</p>
                <p><a href="{spotify_info['spotify_url']}" target="_blank" style="color: #1DB954; text-decoration: none;">Listen on Spotify</a></p>
            </div>
        </div>
        """
        if spotify_info.get('preview_url'):
            html_content += f"<audio controls src='{spotify_info['preview_url']}' style='width: 100%;'></audio>"
        display(HTML(html_content))
    else:
        display(HTML(f"<p>{spotify_info}</p>"))

In [24]:
import ipywidgets as widgets
from IPython.display import display, HTML

custom_css = HTML("""
<style>
    .widget-button .btn { background-color: black; color: white; }
</style>
""")

display(custom_css)

# Create a text input for lyrics
text_input = widgets.Text(
    value='',
    placeholder='Type your lyrics here...',
    description='Query:',
    disabled=False
)

search_button = widgets.Button(
    description='Search Lyrics',
    button_style='',  # This now uses custom CSS for styling
    icon='search'
)

spotify_button = widgets.Button(
    description='Search Spotify',
    button_style='',  # Use custom styles instead of predefined
    icon='spotify'  # Assuming there's a 'spotify' icon in FontAwesome; otherwise, use 'music'
)
spotify_button.style.button_color = 'rgb(30, 215, 96)'

output = widgets.Output()
spotify_output = widgets.Output()

def on_search_button_clicked(b):
    query = text_input.value
    result = search_lyrics_sentence_transformer(query, df, get_openai_embeddings)
    with output:
        output.clear_output()
        first_result = result.iloc[0]
        display(HTML(f"<b>Artist:</b> {first_result['artist']} <br> <b>Title:</b> {first_result['title']} <br> <b>Release Date:</b> {first_result['release_date']}"))
        # Store result for Spotify search
        spotify_button.artist = first_result['artist']
        spotify_button.title = first_result['title']

def on_spotify_button_clicked(b):
    artist = spotify_button.artist
    title = spotify_button.title
    spotify_info = search_spotify_from_results(title, artist)
    with spotify_output:
        spotify_output.clear_output()
        display_spotify_details(spotify_info)  # Use the new visual display function
        

search_button.on_click(on_search_button_clicked)
spotify_button.on_click(on_spotify_button_clicked)

display(text_input, search_button, output, spotify_button, spotify_output)

Text(value='', description='Query:', placeholder='Type your lyrics here...')

Button(description='Search Lyrics', icon='search', style=ButtonStyle())

Output()

Button(description='Search Spotify', icon='spotify', style=ButtonStyle(button_color='rgb(30, 215, 96)'))

Output()